## Цель ноутбука: изучение метода Few Shots Learning

#### В RTSD не хватает 14 знаков:

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | - |
| 1.31 | Туннель | - |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | - |
| 3.24-100 | Огр 100 | GTSRB Recognition |
| 3.24-110 | Огр 110 | - |
| 3.24-120 | Огр 120 | GTSRB Recognition |
| 3.24-130 | Огр 130 | - |
| 3.25 | Конец огр. максимальной скорости | GTSRB Recognition |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | - |

Инициализация библиотек

In [ ]:
import albumentations as A
if A.__version__ != '1.0.3':
    !pip install albumentations==1.0.3
    !pip install opencv-python-headless==4.5.2.52
    assert False, 'restart runtime pls'

import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
import cv2
import sys
from datetime import datetime

TEXT_COLOR = 'black'
# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

IN_COLAB = False
USE_COLAB_GPU = False

try:
    import google.colab
    IN_COLAB = True
    USE_COLAB_GPU = True
    from google.colab import drive
except:
    if IN_COLAB:
        print('[!]YOU ARE IN COLAB, BUT DIDNT MOUND A DRIVE. Model wont be synced[!]')

        if not os.path.isfile(CURRENT_FILE_NAME):
            print("FIX ME")
        IN_COLAB = False

    else:
        pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Инициализация основных путей и папки src

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

SRC_DIR = PROJECT_ROOT / 'src'
sys.path.append(SRC_DIR)

In [ ]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
        
model = Net().to(device)

In [ ]:
model

In [ ]:
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

dataset1 = datasets.MNIST(".", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST(".", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=256)

In [ ]:
test_loader.num_workers

In [ ]:
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester(dataloader_num_workers=0)
    return tester.get_all_embeddings(dataset, model)


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    print('train was called')
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        # print(data)
        # print(labels)
        # input()
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        # print(indices_tuple)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight)
        
# model.apply(init_normal)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 1


### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",))
### pytorch-metric-learning stuff ###


for epoch in range(1, num_epochs + 1):
    # train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

In [ ]:
!pip install plotly

In [ ]:
from tqdm.notebook import trange, tqdm
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from umap import UMAP
import plotly.express as px

@torch.no_grad()
def plotSmth(model, loader, device='cpu', dim3=False, fcn='PCA'):
    model.eval()
    
    fig = plt.figure(figsize=(16, 16))    
    # clean the figure
    fig.clf()
    
    n_components = 3 if dim3 else 2
    
    fcn = fcn.upper()
    
    if fcn == 'PCA':
        reducer = PCA(n_components=n_components, random_state=RANDOM_STATE) 
    elif fcn == 'TSNE':
        reducer = TSNE(n_components=n_components, init='random', random_state=RANDOM_STATE)
    elif fcn == 'UMAP':
        reducer = UMAP(n_components=n_components, init='random', random_state=RANDOM_STATE)
    else:
        assert False, "wrong fcn arg"
            
    # colormap=plt.cm.Paired
    
    pbar = tqdm(enumerate(loader),
                    total=len(loader), 
                    position=0,
                    leave=False)

    MODEL_ARRAY_OUT_SIZE = (0, 128)
    model_out_arr = np.empty(MODEL_ARRAY_OUT_SIZE, dtype=np.float32)
    target_arr = np.empty((0, 1), dtype=np.int32)

    for idx, (data, target) in pbar:

        data = data.to(device)
        out = model(data).detach().cpu().numpy()
        # print(out.shape)
        model_out_arr = np.append(model_out_arr, out, axis=0)
        target = target.detach().cpu().numpy()
        # print(target)
        target_arr = np.append(target_arr, target)

        if idx > 1:
            break

    # print(len(target_arr)) 
    # print(target_arr)
    X_embedded = reducer.fit_transform(model_out_arr)
    target_arr = np.char.mod('%d', target_arr)
    # print(target_arr)
    
    if not dim3:
        fig = px.scatter(X_embedded, x=0, y=1, color=target_arr)
    else:
        fig = px.scatter_3d(X_embedded, x=0, y=1, z=2, color=target_arr)    
    fig.show()
    
    
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=256)

plotSmth(model, test_loader, device=device, dim3=False, fcn='pca')

In [ ]:
plt.show()

In [ ]:
enumerate(train_loader)[0]

In [ ]:
model(dataset1[1][0][None, ...].to(device)).shape

In [ ]:
res = get_all_embeddings(dataset1, model)

In [ ]:
res[0].shape

In [ ]:
from torchvision.transforms.functional import pad

In [ ]:
pad(dataset1[1], 1)

In [ ]:
'''
    This class works under the directory structure of the Omniglot Dataset
    It creates the pairs of images for inputs, same character label = 1, vice versa
'''
class OmniglotDataset(Dataset):
    '''
        categories is the list of different alphabets (folders)
        root_dir is the root directory leading to the alphabet files, could be /images_background or /images_evaluation
        setSize is the size of the train set and the validation set combined
        transform is any image transformations
    '''
    def __init__(self, categories, root_dir, setSize, transform=None):
        self.categories = categories
        self.root_dir = root_dir
        self.transform = transform
        self.setSize = setSize
    def __len__(self):
        return self.setSize
    def __getitem__(self, idx):
        img1 = None
        img2 = None
        label = None
        if idx % 2 == 0: # select the same character for both images
            category = random.choice(categories)
            character = random.choice(category[1])
            imgDir = root_dir + category[0] + '/' + character
            img1Name = random.choice(os.listdir(imgDir))
            img2Name = random.choice(os.listdir(imgDir))
            img1 = Image.open(imgDir + '/' + img1Name)
            img2 = Image.open(imgDir + '/' + img2Name)
            label = 1.0
        else: # select a different character for both images
            category1, category2 = random.choice(categories), random.choice(categories)
            category1, category2 = random.choice(categories), random.choice(categories)
            character1, character2 = random.choice(category1[1]), random.choice(category2[1])
            imgDir1, imgDir2 = root_dir + category1[0] + '/' + character1, root_dir + category2[0] + '/' + character2
            img1Name = random.choice(os.listdir(imgDir1))
            img2Name = random.choice(os.listdir(imgDir2))
            while img1Name == img2Name:
                img2Name = random.choice(os.listdir(imgDir2))
            label = 0.0
            img1 = Image.open(imgDir1 + '/' + img1Name)
            img2 = Image.open(imgDir2 + '/' + img2Name)
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.from_numpy(np.array([label], dtype=np.float32))  

In [ ]:
STOCK_SIGNS_CSV_LOCATION = DATA_DIR / 'STOCK_SIGNS.csv'
STOCK_SIGNS_DATAFRAME = pd.read_csv(STOCK_SIGNS_CSV_LOCATION)

# Исправляем пешеходов в разные стороны
STOCK_SIGNS_DATAFRAME.loc[STOCK_SIGNS_DATAFRAME['SIGN'] == '5.19.2', 'SIGN'] = '5.19.1'

from sklearn import preprocessing

# кодируем знаки, добавляем еще колонку
LE_LOCATION = DATA_DIR / 'STOCK_SIGNS_LE.npy'
le = preprocessing.LabelEncoder()

if os.path.isfile(LE_LOCATION):
    le.classes_ = np.load(LE_LOCATION, allow_pickle=True)
    print('[+] label encoder was restored')
else:
    le.fit_transform(STOCK_SIGNS_DATAFRAME['SIGN'])
    np.save(LE_LOCATION, le.classes_)
    print('[!] label encoder was created')
    
STOCK_SIGNS_DATAFRAME['ENCODED_LABELS'] = le.transform(STOCK_SIGNS_DATAFRAME['SIGN'])
STOCK_SIGNS_DATAFRAME['filepath'] = STOCK_SIGNS_DATAFRAME['filepath'].apply(lambda x: str(x).replace('\\', '/'))
STOCK_SIGNS_DATAFRAME['filepath'] = STOCK_SIGNS_DATAFRAME['filepath'].apply(lambda x: DATA_DIR / x)
STOCK_SIGNS_DATAFRAME[::6]

In [ ]:
SIGN_LIST = list(STOCK_SIGNS_DATAFRAME['SIGN'])

nrows, ncols = 20, 6
fig = plt.figure(figsize = (16,50))

for idx, row in enumerate(STOCK_SIGNS_DATAFRAME.iterrows()):
    instance = row[1]
    path = str(instance['filepath'])
    sign = instance['SIGN']
    encoded_label = instance['ENCODED_LABELS']
    
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    trans_mask = img[:,:,3] == 0
    img[trans_mask] = [random.randrange(0, 256), random.randrange(0, 256), random.randrange(0, 256), 255]
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    ax = fig.add_subplot(nrows, ncols, idx+1)
    
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title('ENCODED: ' + str(encoded_label) 
                 + '\nSIGN: ' + str(sign)
                )
    
plt.tight_layout()